In [33]:
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import pandas as pd
import numpy as np
import torch.nn.functional as F
# 定义超参数
input_size = 2
# 隐藏层大小，一般为输入特征的2~4倍
hidden_size = 64
# LSTM层数，较深的网络可能更能够捕捉序列中的长期依赖关系，但也会增加训练时间和计算成本。
# 通常建议使用 1-3 层 LSTM，具体取决于数据集和问题复杂度。
num_layers = 1
output_size = 1
batch_size = 16
# 通常建议使用较小的学习率（例如 0.001）进行初始训练，然后根据模型在验证集上的性能进行微调。
learning_rate = 0.002
num_epochs = 10

In [34]:
class MyDataset(Dataset):
    def __init__(self, data_path, sequence_length):
        self.data = pd.read_csv(data_path)
        self.sequence_length = sequence_length
        self.features = self.data.drop(['write_counts'], axis=1).values.astype(np.float32)
        self.targets = self.data['write_counts'].values.astype(np.float32)
    
    def __len__(self):
        return len(self.data) - self.sequence_length
    
    def __getitem__(self, idx):
        start_idx = idx
        end_idx = idx + self.sequence_length
        x = self.features[start_idx:end_idx]

        # 手动归一化
        x = torch.from_numpy(x)
        x = F.normalize(x, dim=0)  # 在维度0上进行归一化

        y = self.targets[end_idx]
        return x, y

In [35]:
""" 
一般情况下，LSTM 模型的输入应该是一个 3 维张量，其维度为 [batch_size, sequence_length, input_size]，
分别表示批次大小、时间序列长度和输入维度。因此，在处理数据集时，需要将原始数据转换成这种形式。
"""
class MyLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(MyLSTM, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
        
    def forward(self, x):
        # 将输入的形状从 [batch_size * sequence_length, input_size] 转换为 [batch_size, sequence_length, input_size]
        # batch_size = x.size(0) // self.num_layers
        # x = x.view(batch_size, -1, self.lstm.input_size)

        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        out, _ = self.lstm(x, (h0, c0))
        out = out[:, -1, :]
        out = self.fc(out)
        return out
# class LSTM(nn.Module):
#     def __init__(self):
#         super(LSTM, self).__init__()
        
#         self.lstm = nn.LSTM(
#             input_size=1,   # 输入尺寸为 1，表示一天的数据
#             hidden_size=64,
#             num_layers=1, 
#             batch_first=True)
        
#         self.out = nn.Sequential(
#             nn.Linear(64,1))
        
#     def forward(self, x):
#         r_out, (h_n, h_c) = self.lstm(x, None)   # None 表示 hidden state 会用全 0 的 state
#         out = self.out(r_out[:, -1, :])          # 取最后一天作为输出
#         out = self.fc(out)
#         return out

In [36]:
# 训练集路径
train_data_path = "./dataset/btcusd_low.csv"
train_ratio = 0.7
# 加载数据集
dataset = MyDataset(train_data_path, sequence_length=32)
train_size = int(len(dataset) * train_ratio)
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

# 加载数据并训练模型
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)
# 定义模型、损失函数和优化器
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = MyLSTM(input_size, hidden_size, num_layers, output_size)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [37]:
for epoch in range(num_epochs):
    for i, (inputs, targets) in enumerate(train_loader):
        inputs = inputs.float()
        targets = targets.float()

        # 前向传播
        outputs = model(inputs)
        loss = criterion(outputs, targets)

        # 反向传播和优化
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % 200 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'
                  .format(epoch+1, num_epochs, i+1, len(train_loader), loss.item()))


/home/jason/.local/lib/python3.8/site-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([16])) that is different to the input size (torch.Size([16, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch [1/10], Step [200/1312], Loss: 4.5745
Epoch [1/10], Step [400/1312], Loss: 3.0478
Epoch [1/10], Step [600/1312], Loss: 5.7233
Epoch [1/10], Step [800/1312], Loss: 6.7207
Epoch [1/10], Step [1000/1312], Loss: 6.3749
Epoch [1/10], Step [1200/1312], Loss: 2.8344


/home/jason/.local/lib/python3.8/site-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch [2/10], Step [200/1312], Loss: 5.4652
Epoch [2/10], Step [400/1312], Loss: 3.5360
Epoch [2/10], Step [600/1312], Loss: 4.4844
Epoch [2/10], Step [800/1312], Loss: 3.2595
Epoch [2/10], Step [1000/1312], Loss: 5.5353
Epoch [2/10], Step [1200/1312], Loss: 5.1037
Epoch [3/10], Step [200/1312], Loss: 8.0051
Epoch [3/10], Step [400/1312], Loss: 2.9200
Epoch [3/10], Step [600/1312], Loss: 1.7765
Epoch [3/10], Step [800/1312], Loss: 6.2732
Epoch [3/10], Step [1000/1312], Loss: 4.7934
Epoch [3/10], Step [1200/1312], Loss: 3.3206
Epoch [4/10], Step [200/1312], Loss: 3.3787
Epoch [4/10], Step [400/1312], Loss: 2.1205
Epoch [4/10], Step [600/1312], Loss: 4.8667
Epoch [4/10], Step [800/1312], Loss: 7.6359
Epoch [4/10], Step [1000/1312], Loss: 7.2767
Epoch [4/10], Step [1200/1312], Loss: 7.6587
Epoch [5/10], Step [200/1312], Loss: 5.3630
Epoch [5/10], Step [400/1312], Loss: 3.2955
Epoch [5/10], Step [600/1312], Loss: 5.4851
Epoch [5/10], Step [800/1312], Loss: 3.4741
Epoch [5/10], Step [1000/1

In [38]:
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for inputs, targets in test_loader:
        inputs = inputs.float()
        # print(inputs)
        targets = targets.float()
        # print(targets)

        outputs = model(inputs)
        predicted = torch.squeeze(torch.round(outputs))
        print(predicted)
        total += targets.size(0)
        temp = predicted == targets
        correct += temp.sum().item()

    accuracy = 100 * correct / total
    print('Accuracy of the model on the test dataset: {:.2f} %'.format(accuracy))

tensor([6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6.])
tensor([6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6.])
tensor([6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6.])
tensor([6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6.])
tensor([6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6.])
tensor([6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6.])
tensor([6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6.])
tensor([6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6.])
tensor([6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6.])
tensor([6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6.])
tensor([6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6.])
tensor([6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6.])
tensor([6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6.])
tensor([6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6.,

In [39]:
print(train_dataset.dataset.data)

          read_time  write_counts      period
0      1.364778e+12           6.0         0.0
1      1.364780e+12           5.0   2580000.0
2      1.364785e+12           3.0   4560000.0
3      1.364797e+12           5.0  12660000.0
4      1.364811e+12           6.0  13320000.0
...             ...           ...         ...
29995  1.404257e+12           4.0    600000.0
29996  1.404258e+12           6.0    660000.0
29997  1.404259e+12           4.0    600000.0
29998  1.404259e+12           8.0    600000.0
29999  1.404260e+12           7.0    600000.0

[30000 rows x 3 columns]


In [40]:
mean_values = np.mean(dataset.features, axis=0)
std_dev_values = np.std(dataset.features, axis=0)

print("Mean values for each feature:", mean_values)
print("Standard deviation for each feature:", std_dev_values)

Mean values for each feature: [1.3873809e+12 1.3160740e+06]
Standard deviation for each feature: [1.0782867e+10 1.3323338e+06]


In [41]:
normalized_features = (dataset.features - mean_values) / std_dev_values

# 计算归一化后的均值和标准差
normalized_mean_values = np.mean(normalized_features, axis=0)
normalized_std_dev_values = np.std(normalized_features, axis=0)

print("Normalized Mean values for each feature:", normalized_mean_values)
print("Normalized Standard deviation for each feature:", normalized_std_dev_values)

Normalized Mean values for each feature: [2.5390625e-06 1.8310548e-08]
Normalized Standard deviation for each feature: [1. 1.]
